In [ ]:
!pip -q install ultralytics==8.* opencv-python matplotlib grad-cam

import os, glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Torch: 2.9.0+cu126
CUDA available: True


In [ ]:
DATA_DIR = "/content/images"   # <- change this (folder has best.pt and images)
WEIGHTS  = os.path.join(DATA_DIR, "hybrid_wbf.pt")

OUT_DIR  = "/content/outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# Collect images
img_exts = ("*.jpg","*.jpeg","*.png","*.webp")
image_paths = []
for e in img_exts:
    image_paths += glob.glob(os.path.join(DATA_DIR, e))
image_paths = sorted(image_paths)

print("Weights:", WEIGHTS, "exists:", os.path.exists(WEIGHTS))
print("Found images:", len(image_paths))
for p in image_paths:
    print(" -", p)

Weights: /content/images/hybrid_wbf.pt exists: True
Found images: 2
 - /content/images/emotion_female.png
 - /content/images/emotion_male.png


In [ ]:
!pip install ensemble-boxes

In [ ]:
import os
import torch
from ultralytics import YOLO
import cv2
import numpy as np
from ensemble_boxes import weighted_boxes_fusion

# ----------------------------
# Paths
# ----------------------------
WEIGHTS = "/content/hybrid_out/hybrid_wbf.pt"
OUT_DIR = "/content/hybrid_out"
pred_dir = os.path.join(OUT_DIR, "pred")
os.makedirs(pred_dir, exist_ok=True)

# image_paths = [...]  # list of image paths

# ----------------------------
# Load hybrid checkpoint (NOT via YOLO)
# ----------------------------
ckpt = torch.load(WEIGHTS, map_location="cpu")

# ✅ CORRECT KEYS
w8  = ckpt["yolov8_weight_path"]
w11 = ckpt["yolov11_weight_path"]

fusion_cfg = ckpt["fusion"]
wbf_weights = tuple(fusion_cfg.get("weights", (1.0, 1.0)))
wbf_iou     = float(fusion_cfg.get("iou_thr", 0.55))
wbf_skip    = float(fusion_cfg.get("skip_box_thr", 0.001))

# ----------------------------
# Load YOLO models
# ----------------------------
device = 0  # GPU
m8  = YOLO(w8)
m11 = YOLO(w11)

names = m8.names  # original class names

# ----------------------------
# Helpers
# ----------------------------
def yolo_to_norm(res, w, h):
    if res.boxes is None or len(res.boxes) == 0:
        return [], [], []
    boxes = res.boxes.xyxy.detach().cpu().numpy()
    scores = res.boxes.conf.detach().cpu().numpy()
    labels = res.boxes.cls.detach().cpu().numpy().astype(int)

    boxes[:, [0, 2]] /= w
    boxes[:, [1, 3]] /= h
    boxes = np.clip(boxes, 0.0, 1.0)
    return boxes.tolist(), scores.tolist(), labels.tolist()

def fuse_wbf(res8, res11, w, h):
    b8, s8, l8 = yolo_to_norm(res8, w, h)
    b11, s11, l11 = yolo_to_norm(res11, w, h)

    boxes, scores, labels = weighted_boxes_fusion(
        [b8, b11],
        [s8, s11],
        [l8, l11],
        weights=list(wbf_weights),
        iou_thr=wbf_iou,
        skip_box_thr=wbf_skip
    )

    boxes = np.array(boxes)
    scores = np.array(scores)
    labels = np.array(labels, dtype=int)

    if len(boxes) > 0:
        boxes[:, [0, 2]] *= w
        boxes[:, [1, 3]] *= h

    return boxes, scores, labels

def draw_boxes(img, boxes, scores, labels, conf_thr=0.25):
    out = img.copy()
    for (x1, y1, x2, y2), sc, lb in zip(boxes, scores, labels):
        if sc < conf_thr:
            continue
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        name = names[lb] if isinstance(names, dict) else names[lb]
        cv2.rectangle(out, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(out, f"{name} {sc:.2f}", (x1, max(15, y1 - 5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return out

# ----------------------------
# Run Hybrid Prediction
# ----------------------------
for p in image_paths:
    img = cv2.imread(p)
    if img is None:
        continue
    h, w = img.shape[:2]

    r8  = m8.predict(p, imgsz=640, conf=0.001, device=device, verbose=False)[0]
    r11 = m11.predict(p, imgsz=640, conf=0.001, device=device, verbose=False)[0]

    boxes, scores, labels = fuse_wbf(r8, r11, w, h)

    vis = draw_boxes(img, boxes, scores, labels)

    out_path = os.path.join(pred_dir, os.path.splitext(os.path.basename(p))[0] + ".jpg")
    cv2.imwrite(out_path, vis)

print("✅ Saved HYBRID predictions into:", pred_dir)

✅ Saved HYBRID predictions into: /content/hybrid_out/pred


In [ ]:
import os, cv2, torch
import numpy as np
import torch.nn as nn
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image

# ============================================================
# Hybrid Grad-CAM++ (YOLOv8 + YOLOv11) → fused heatmap
# Requires: m8, m11 already loaded as ultralytics.YOLO objects
# Requires: image_paths (list of image paths), OUT_DIR (output root)
# ============================================================

device = "cuda" if torch.cuda.is_available() else "cpu"

# ---- helper: find a good conv layer automatically (last Conv2d with spatial map > 1x1) ----
def find_last_good_conv2d(torch_model, input_shape=(1,3,640,640), device="cuda"):
    acts, hooks = [], []

    def hook_fn(m, inp, out):
        if isinstance(out, torch.Tensor) and out.ndim == 4:
            acts.append((m, out.shape))  # (module, [B,C,H,W])

    for m in torch_model.modules():
        if isinstance(m, nn.Conv2d):
            hooks.append(m.register_forward_hook(hook_fn))

    with torch.no_grad():
        dummy = torch.zeros(*input_shape, device=device)
        _ = torch_model(dummy)

    for h in hooks:
        h.remove()

    good = [x for x in acts if x[1][2] > 1 and x[1][3] > 1]  # H,W > 1
    return good[-1][0] if good else (acts[-1][0] if acts else None)

# ---- helper: letterbox resize like YOLO ----
def letterbox(im, new_shape=640, color=(114,114,114)):
    h, w = im.shape[:2]
    r = min(new_shape / h, new_shape / w)
    nh, nw = int(round(h * r)), int(round(w * r))
    im_resized = cv2.resize(im, (nw, nh), interpolation=cv2.INTER_LINEAR)
    canvas = np.full((new_shape, new_shape, 3), color, dtype=np.uint8)
    top = (new_shape - nh) // 2
    left = (new_shape - nw) // 2
    canvas[top:top+nh, left:left+nw] = im_resized
    return canvas

def to_tensor_yolo(img_bgr_640, device="cuda"):
    img_rgb = cv2.cvtColor(img_bgr_640, cv2.COLOR_BGR2RGB)
    img_float = img_rgb.astype(np.float32) / 255.0
    x = torch.from_numpy(img_float).permute(2,0,1).unsqueeze(0).to(device)  # [1,3,640,640]
    return x, img_float  # tensor, rgb_float

# ---- IMPORTANT: use the raw torch models for gradient flow ----
torch_m8  = m8.model.to(device).eval()
torch_m11 = m11.model.to(device).eval()

# ---- pick CAM layers for both models ----
layer8  = find_last_good_conv2d(torch_m8,  input_shape=(1,3,640,640), device=device)
layer11 = find_last_good_conv2d(torch_m11, input_shape=(1,3,640,640), device=device)
if layer8 is None or layer11 is None:
    raise RuntimeError("Could not find suitable Conv2d layer(s) for CAM.")

print("YOLOv8 CAM layer :", layer8)
print("YOLOv11 CAM layer:", layer11)

cam8  = GradCAMPlusPlus(model=torch_m8,  target_layers=[layer8])
cam11 = GradCAMPlusPlus(model=torch_m11, target_layers=[layer11])

cam_dir = os.path.join(OUT_DIR, "hybrid_gradcampp")
os.makedirs(cam_dir, exist_ok=True)

# ---- CAM target that DEPENDS on model output (keeps grad graph) ----
class YoloMaxScoreTarget:
    def __call__(self, model_output):
        # unwrap nested outputs
        if isinstance(model_output, (list, tuple)):
            model_output = model_output[0]
            if isinstance(model_output, (list, tuple)):
                model_output = model_output[0]

        if not torch.is_tensor(model_output):
            raise RuntimeError("Model output is not a torch.Tensor")

        if model_output.ndim == 2:
            model_output = model_output.unsqueeze(0)  # [1, N, D]

        d = model_output.shape[-1]
        if d <= 6:
            return model_output.max()

        score_a = model_output[..., 4:].max()
        obj = model_output[..., 4]
        if d > 5:
            cls = model_output[..., 5:]
            score_b = (obj.unsqueeze(-1) * cls).max()
        else:
            score_b = obj.max()

        return torch.maximum(score_a, score_b)

# ---- hybrid CAM fusion weights (use your WBF weights if available) ----
try:
    w8_cam, w11_cam = float(wbf_weights[0]), float(wbf_weights[1])  # if you defined wbf_weights earlier
except Exception:
    w8_cam, w11_cam = 1.0, 1.0
w_sum = w8_cam + w11_cam + 1e-9

# ---- run HYBRID CAM for each image ----
for i, img_path in enumerate(image_paths):
    bgr = cv2.imread(img_path)
    if bgr is None:
        print("Skip (cannot read):", img_path)
        continue

    bgr640 = letterbox(bgr, new_shape=640)
    x, rgb_float = to_tensor_yolo(bgr640, device=device)

    x.requires_grad_(True)
    torch.set_grad_enabled(True)

    cam_map8  = cam8(input_tensor=x,  targets=[YoloMaxScoreTarget()])[0]   # [640,640]
    cam_map11 = cam11(input_tensor=x, targets=[YoloMaxScoreTarget()])[0]   # [640,640]

    hybrid_cam = (w8_cam * cam_map8 + w11_cam * cam_map11) / w_sum

    overlay = show_cam_on_image(rgb_float, hybrid_cam, use_rgb=True)
    overlay_bgr = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)

    out_name = os.path.splitext(os.path.basename(img_path))[0] + "_HYBRID_gradcampp.jpg"
    out_path = os.path.join(cam_dir, out_name)
    cv2.imwrite(out_path, overlay_bgr)

    print(f"[{i+1}/{len(image_paths)}] Saved:", out_path)

print("✅ Hybrid Grad-CAM++ outputs in:", cam_dir)

YOLOv8 CAM layer : Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
YOLOv11 CAM layer: Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
[1/2] Saved: /content/hybrid_out/hybrid_gradcampp/emotion_female_HYBRID_gradcampp.jpg
[2/2] Saved: /content/hybrid_out/hybrid_gradcampp/emotion_male_HYBRID_gradcampp.jpg
✅ Hybrid Grad-CAM++ outputs in: /content/hybrid_out/hybrid_gradcampp


In [ ]:
import glob
cams = sorted(glob.glob("/content/hybrid_out/hybrid_gradcampp*.jpg"))
preds = sorted(glob.glob("/content/hybrid_out/pred*.jpg"))

print("CAMs:", len(cams))
for p in cams:
    img = cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8,6))
    plt.imshow(img)
    plt.axis("off")
    plt.title(os.path.basename(p))
    plt.show()

CAMs: 0


In [ ]:
# ============================================================
# Explainability Utilities for YOLO / Hybrid YOLO (clean rewrite)
# Installs, imports, helper funcs, and faithfulness metrics:
# - Occlusion Sensitivity Map
# - Deletion / Insertion Curves
# ============================================================
import os, glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO
from pytorch_grad_cam.utils.image import show_cam_on_image

# -------------------------
# CONFIG
# -------------------------
OUT_BASE = "/content/outputs/explainability_plots"
os.makedirs(OUT_BASE, exist_ok=True)

# If you already defined `model` and `image_paths`, keep them.
# Example:
# model = YOLO("/content/best.pt")
# image_paths = sorted(glob.glob("/content/test_imgs/*.jpg"))

# -------------------------
# Filesystem helpers
# -------------------------
def ensure_dir(path: str) -> str:
    os.makedirs(path, exist_ok=True)
    return path

def save_bgr(path: str, bgr: np.ndarray) -> None:
    cv2.imwrite(path, bgr)

def save_fig(path: str) -> None:
    plt.savefig(path, dpi=200, bbox_inches="tight")
    plt.close()

# -------------------------
# Preprocess helpers
# -------------------------
def letterbox(im: np.ndarray, new_shape: int = 640, color=(114,114,114)) -> np.ndarray:
    """Square letterbox resize (YOLO-style) to new_shape x new_shape."""
    h, w = im.shape[:2]
    r = min(new_shape / h, new_shape / w)
    nh, nw = int(round(h * r)), int(round(w * r))
    im_resized = cv2.resize(im, (nw, nh), interpolation=cv2.INTER_LINEAR)

    canvas = np.full((new_shape, new_shape, 3), color, dtype=np.uint8)
    top = (new_shape - nh) // 2
    left = (new_shape - nw) // 2
    canvas[top:top+nh, left:left+nw] = im_resized
    return canvas

def bgr_to_rgb01(bgr: np.ndarray) -> np.ndarray:
    """BGR uint8 -> RGB float32 in [0,1]."""
    return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

# -------------------------
# Visualization helpers
# -------------------------
def draw_yolo_boxes(bgr: np.ndarray, res, color=(0,255,0), thickness: int = 2) -> np.ndarray:
    """Draw Ultralytics YOLO boxes on a BGR image."""
    out = bgr.copy()
    if res is None or res.boxes is None or len(res.boxes) == 0:
        return out

    xyxy = res.boxes.xyxy.detach().cpu().numpy().astype(int)
    conf = res.boxes.conf.detach().cpu().numpy()
    cls  = res.boxes.cls.detach().cpu().numpy().astype(int)
    names = getattr(res, "names", None)

    for (x1,y1,x2,y2), c, k in zip(xyxy, conf, cls):
        cv2.rectangle(out, (x1,y1), (x2,y2), color, thickness)
        cname = names[k] if (isinstance(names, dict) and k in names) else (names[k] if isinstance(names, list) else str(k))
        cv2.putText(out, f"{cname}: {c:.2f}", (x1, max(15, y1-5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return out

def normalize01(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32)
    x = x - x.min()
    return x / (x.max() - x.min() + 1e-9)

def cam_to_heatmap_bgr(cam_2d: np.ndarray) -> np.ndarray:
    """2D CAM -> BGR heatmap (JET)."""
    cam01 = (normalize01(cam_2d) * 255).astype(np.uint8)
    return cv2.applyColorMap(cam01, cv2.COLORMAP_JET)

def contour_overlay(bgr: np.ndarray, cam_2d: np.ndarray, levels=(0.4, 0.6, 0.8)) -> np.ndarray:
    """Overlay contours at CAM thresholds."""
    cam01 = normalize01(cam_2d)
    out = bgr.copy()
    for lv in levels:
        mask = (cam01 >= lv).astype(np.uint8) * 255
        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(out, cnts, -1, (0,0,255), 2)
    return out

def threshold_mask(cam_2d: np.ndarray, thr: float = 0.6) -> np.ndarray:
    """Binary mask from CAM."""
    return (normalize01(cam_2d) >= thr).astype(np.uint8)

def montage3(rgb_list, titles, out_path: str) -> None:
    """Save a 1x3 montage (RGB images expected)."""
    plt.figure(figsize=(15,5))
    for i, (im, t) in enumerate(zip(rgb_list, titles), 1):
        plt.subplot(1,3,i)
        plt.imshow(im)
        plt.title(t)
        plt.axis("off")
    save_fig(out_path)

# ============================================================
# Faithfulness scoring for YOLO / Hybrid YOLO
# (Uses model.predict; suitable for occlusion + del/ins curves.)
# ============================================================

def yolo_top_score(model, img_bgr: np.ndarray, imgsz: int = 640, conf: float = 0.25, device=0) -> float:
    """Return top confidence among all predicted boxes for an image."""
    r = model.predict(source=img_bgr, imgsz=imgsz, conf=conf, device=device, verbose=False)[0]
    if r.boxes is None or len(r.boxes) == 0:
        return 0.0
    return float(r.boxes.conf.max().detach().cpu().item())

def occlusion_sensitivity_map(model,
                             img_bgr: np.ndarray,
                             patch: int = 64,
                             stride: int = 64,
                             imgsz: int = 640,
                             conf: float = 0.25,
                             device=0) -> np.ndarray:
    """
    Occlusion sensitivity:
    - compute base score
    - black-out patches and measure score drop
    - upsample to image size
    Returns: HxW float map in [0,1]
    """
    base = yolo_top_score(model, img_bgr, imgsz=imgsz, conf=conf, device=device)
    H, W = img_bgr.shape[:2]

    oh = int(np.ceil((H - patch) / stride)) + 1
    ow = int(np.ceil((W - patch) / stride)) + 1
    m = np.zeros((oh, ow), dtype=np.float32)

    for yi in range(oh):
        for xi in range(ow):
            y1, x1 = yi * stride, xi * stride
            y2, x2 = min(H, y1 + patch), min(W, x1 + patch)

            masked = img_bgr.copy()
            masked[y1:y2, x1:x2] = 0  # black mask

            s = yolo_top_score(model, masked, imgsz=imgsz, conf=conf, device=device)
            m[yi, xi] = max(0.0, base - s)

    m = normalize01(m)
    return cv2.resize(m, (W, H), interpolation=cv2.INTER_CUBIC)

def deletion_insertion_curves(model,
                              img_bgr: np.ndarray,
                              importance_map: np.ndarray,
                              steps: int = 20,
                              imgsz: int = 640,
                              conf: float = 0.25,
                              device=0):
    """
    Deletion: progressively remove most important pixels (set to 0).
    Insertion: start blurred, progressively insert important pixels.
    Returns: fracs, del_scores, ins_scores
    """
    H, W = img_bgr.shape[:2]
    imp = importance_map.reshape(-1)
    order = np.argsort(-imp)  # most important first

    rgb = bgr_to_rgb01(img_bgr)
    flat = rgb.reshape(-1, 3)

    blur = cv2.GaussianBlur(img_bgr, (31,31), 0)
    blur_rgb = bgr_to_rgb01(blur)
    flat_blur = blur_rgb.reshape(-1, 3)

    del_scores, ins_scores, fracs = [], [], []
    n = flat.shape[0]

    for t in range(steps + 1):
        frac = t / steps
        k = int(frac * n)

        # Deletion
        del_flat = flat.copy()
        del_flat[order[:k]] = 0.0
        del_img = (del_flat.reshape(H, W, 3) * 255).astype(np.uint8)
        del_bgr = cv2.cvtColor(del_img, cv2.COLOR_RGB2BGR)
        del_scores.append(yolo_top_score(model, del_bgr, imgsz=imgsz, conf=conf, device=device))

        # Insertion
        ins_flat = flat_blur.copy()
        ins_flat[order[:k]] = flat[order[:k]]
        ins_img = (ins_flat.reshape(H, W, 3) * 255).astype(np.uint8)
        ins_bgr = cv2.cvtColor(ins_img, cv2.COLOR_RGB2BGR)
        ins_scores.append(yolo_top_score(model, ins_bgr, imgsz=imgsz, conf=conf, device=device))

        fracs.append(frac)

    return np.array(fracs), np.array(del_scores), np.array(ins_scores)

In [ ]:
# ============================================================
# Hybrid Explainability Post-processing & Visualization (ONE BLOCK)
# Uses:
#  - Hybrid Grad-CAM++ overlays: /content/hybrid_out/hybrid_gradcampp
#  - Hybrid/YOLO annotated preds: /content/hybrid_out/pred (optional)
# Produces per-image explainability assets:
#  - Original, Pred annotated, Hybrid CAM overlay + derived plots
#  - Occlusion sensitivity heatmap + overlay
#  - Deletion/Insertion faithfulness curves
# Saves into:
#  - /content/hybrid_out/explainability_plots/<image_name>/
#
# REQUIREMENTS (must already exist in notebook):
#   - image_paths: list of image file paths
#   - m8, m11: ultralytics.YOLO objects (best_y8.pt, best_y11.pt)
#   - If you want hybrid scoring/prediction: ensemble-boxes installed
# ============================================================

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from ultralytics import YOLO
from ensemble_boxes import weighted_boxes_fusion

# -----------------------------
# PATHS (as you requested)
# -----------------------------
CAM_DIR_EXISTING = "/content/hybrid_out/hybrid_gradcampp"
PRED_DIR         = "/content/hybrid_out/pred"
OUT_BASE         = "/content/hybrid_out/explainability_plots"
os.makedirs(OUT_BASE, exist_ok=True)

# -----------------------------
# Small utils (self-contained)
# -----------------------------
def ensure_dir(path: str) -> str:
    os.makedirs(path, exist_ok=True)
    return path

def save_img(path: str, bgr: np.ndarray) -> None:
    cv2.imwrite(path, bgr)

def save_matplotlib_figure(path: str) -> None:
    plt.savefig(path, dpi=200, bbox_inches="tight")
    plt.close()

def normalize_0_1(x: np.ndarray) -> np.ndarray:
    x = x.astype(np.float32)
    x = x - x.min()
    return x / (x.max() - x.min() + 1e-9)

def heatmap_bgr_from_cam(cam_2d: np.ndarray) -> np.ndarray:
    cam01 = (normalize_0_1(cam_2d) * 255).astype(np.uint8)
    return cv2.applyColorMap(cam01, cv2.COLORMAP_JET)  # BGR

def threshold_mask(cam_2d: np.ndarray, thr: float = 0.6) -> np.ndarray:
    cam01 = normalize_0_1(cam_2d)
    return (cam01 >= thr).astype(np.uint8)

def contour_overlay(bgr: np.ndarray, cam_2d: np.ndarray, levels=(0.4, 0.6, 0.8)) -> np.ndarray:
    cam01 = normalize_0_1(cam_2d)
    out = bgr.copy()
    for lv in levels:
        mask = (cam01 >= lv).astype(np.uint8) * 255
        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(out, cnts, -1, (0, 0, 255), 2)
    return out

def montage3(rgb_list, titles, out_path: str) -> None:
    plt.figure(figsize=(15, 5))
    for i, (im, t) in enumerate(zip(rgb_list, titles), 1):
        plt.subplot(1, 3, i)
        plt.imshow(im)
        plt.title(t)
        plt.axis("off")
    save_matplotlib_figure(out_path)

def draw_yolo_boxes(bgr: np.ndarray, xyxy: np.ndarray, conf: np.ndarray, cls: np.ndarray, names, color=(0,255,0), thickness=2):
    out = bgr.copy()
    for (x1,y1,x2,y2), c, k in zip(xyxy, conf, cls):
        x1,y1,x2,y2 = map(int, [x1,y1,x2,y2])
        cv2.rectangle(out, (x1,y1), (x2,y2), color, thickness)
        cname = names[k] if isinstance(names, dict) else names[k]
        cv2.putText(out, f"{cname}: {c:.2f}", (x1, max(15, y1-5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return out

# -----------------------------
# HYBRID model wrapper (WBF)
# -----------------------------
class HybridYOLO:
    """
    Hybrid YOLO (YOLOv8 + YOLOv11) with Weighted Boxes Fusion.
    Provides:
      - predict(): returns a lightweight 'result' object similar enough for our needs
      - top_score(): scalar for faithfulness metrics
    """
    def __init__(self, m8, m11, wbf_weights=(1.0, 1.0), iou_thr=0.55, skip_thr=0.001):
        self.m8 = m8
        self.m11 = m11
        self.wbf_weights = tuple(wbf_weights)
        self.iou_thr = float(iou_thr)
        self.skip_thr = float(skip_thr)
        self.names = getattr(m8, "names", None)
        self.device = 0  # Colab GPU id

    def _res_to_norm(self, res, w, h):
        if res.boxes is None or len(res.boxes) == 0:
            return [], [], []
        boxes = res.boxes.xyxy.detach().cpu().numpy().astype(np.float32)
        scores = res.boxes.conf.detach().cpu().numpy().astype(np.float32)
        labels = res.boxes.cls.detach().cpu().numpy().astype(int)

        boxes[:, [0,2]] /= float(w)
        boxes[:, [1,3]] /= float(h)
        boxes = np.clip(boxes, 0.0, 1.0)
        return boxes.tolist(), scores.tolist(), labels.tolist()

    def _fuse(self, res8, res11, w, h):
        b8, s8, l8   = self._res_to_norm(res8,  w, h)
        b11, s11, l11 = self._res_to_norm(res11, w, h)

        boxes, scores, labels = weighted_boxes_fusion(
            [b8, b11],
            [s8, s11],
            [l8, l11],
            weights=list(self.wbf_weights),
            iou_thr=self.iou_thr,
            skip_box_thr=self.skip_thr
        )

        boxes = np.asarray(boxes, dtype=np.float32)
        scores = np.asarray(scores, dtype=np.float32)
        labels = np.asarray(labels, dtype=int)

        if boxes.size:
            boxes[:, [0,2]] *= float(w)
            boxes[:, [1,3]] *= float(h)
        return boxes, scores, labels

    def predict(self, source, imgsz=640, conf=0.25, device=0, verbose=False):
        # source can be path or BGR ndarray
        if isinstance(source, str):
            bgr = cv2.imread(source)
            if bgr is None:
                return [self._empty_result()]
        else:
            bgr = source

        h, w = bgr.shape[:2]

        r8  = self.m8.predict(source=bgr, imgsz=imgsz, conf=0.001, device=device, verbose=verbose)[0]
        r11 = self.m11.predict(source=bgr, imgsz=imgsz, conf=0.001, device=device, verbose=verbose)[0]

        boxes, scores, labels = self._fuse(r8, r11, w, h)

        # apply final conf threshold
        keep = scores >= float(conf)
        boxes = boxes[keep]
        scores = scores[keep]
        labels = labels[keep]

        return [self._wrap_result(boxes, scores, labels, names=self.names)]

    def top_score(self, source, imgsz=640, conf=0.25, device=0):
        res = self.predict(source=source, imgsz=imgsz, conf=conf, device=device, verbose=False)[0]
        if res.boxes is None or len(res.boxes) == 0:
            return 0.0
        return float(res.boxes.conf.max())

    # --- lightweight result structure compatible with our drawing ---
    class _Boxes:
        def __init__(self, xyxy, conf, cls):
            self.xyxy = xyxy
            self.conf = conf
            self.cls  = cls
        def __len__(self):
            return 0 if self.xyxy is None else len(self.xyxy)

    class _Result:
        def __init__(self, boxes, names):
            self.boxes = boxes
            self.names = names

    def _wrap_result(self, xyxy, conf, cls, names):
        return HybridYOLO._Result(
            HybridYOLO._Boxes(xyxy, conf, cls),
            names
        )

    def _empty_result(self):
        return HybridYOLO._Result(HybridYOLO._Boxes(None, None, None), self.names)

# -----------------------------
# Faithfulness metrics (hybrid)
# -----------------------------
def yolo_top_score(model_like, img_bgr, imgsz=640, conf=0.25, device=0):
    # works for HybridYOLO (has top_score) and Ultralytics YOLO (predict)
    if hasattr(model_like, "top_score"):
        return model_like.top_score(img_bgr, imgsz=imgsz, conf=conf, device=device)
    r = model_like.predict(source=img_bgr, imgsz=imgsz, conf=conf, device=device, verbose=False)[0]
    if r.boxes is None or len(r.boxes) == 0:
        return 0.0
    return float(np.max(r.boxes.conf))

def occlusion_sensitivity_map(model_like, img_bgr, patch=64, stride=64, imgsz=640, conf=0.25, device=0):
    base = yolo_top_score(model_like, img_bgr, imgsz=imgsz, conf=conf, device=device)
    H, W = img_bgr.shape[:2]
    oh = int(np.ceil((H - patch) / stride)) + 1
    ow = int(np.ceil((W - patch) / stride)) + 1
    m = np.zeros((oh, ow), dtype=np.float32)

    for yi in range(oh):
        for xi in range(ow):
            y1, x1 = yi * stride, xi * stride
            y2, x2 = min(H, y1 + patch), min(W, x1 + patch)
            masked = img_bgr.copy()
            masked[y1:y2, x1:x2] = 0
            s = yolo_top_score(model_like, masked, imgsz=imgsz, conf=conf, device=device)
            m[yi, xi] = max(0.0, base - s)

    m = normalize_0_1(m)
    return cv2.resize(m, (W, H), interpolation=cv2.INTER_CUBIC)

def deletion_insertion_curves(model_like, img_bgr, importance_map, steps=20, imgsz=640, conf=0.25, device=0):
    H, W = img_bgr.shape[:2]
    imp = importance_map.reshape(-1)
    order = np.argsort(-imp)

    rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    flat = rgb.reshape(-1, 3)

    blur = cv2.GaussianBlur(img_bgr, (31, 31), 0)
    blur_rgb = cv2.cvtColor(blur, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    flat_blur = blur_rgb.reshape(-1, 3)

    del_scores, ins_scores, fracs = [], [], []
    n = flat.shape[0]

    for t in range(steps + 1):
        frac = t / steps
        k = int(frac * n)

        # deletion
        del_flat = flat.copy()
        del_flat[order[:k]] = 0.0
        del_img = (del_flat.reshape(H, W, 3) * 255).astype(np.uint8)
        del_bgr = cv2.cvtColor(del_img, cv2.COLOR_RGB2BGR)
        del_scores.append(yolo_top_score(model_like, del_bgr, imgsz=imgsz, conf=conf, device=device))

        # insertion
        ins_flat = flat_blur.copy()
        ins_flat[order[:k]] = flat[order[:k]]
        ins_img = (ins_flat.reshape(H, W, 3) * 255).astype(np.uint8)
        ins_bgr = cv2.cvtColor(ins_img, cv2.COLOR_RGB2BGR)
        ins_scores.append(yolo_top_score(model_like, ins_bgr, imgsz=imgsz, conf=conf, device=device))

        fracs.append(frac)

    return np.array(fracs), np.array(del_scores), np.array(ins_scores)

# -----------------------------
# Define HYBRID model for this script
# (If you already have wbf_weights from earlier, it will use it;
#  otherwise defaults to (1.0, 1.0))
# -----------------------------
try:
    _wbf_weights = wbf_weights  # if you defined earlier
except Exception:
    _wbf_weights = (1.0, 1.0)

model = HybridYOLO(m8, m11, wbf_weights=_wbf_weights, iou_thr=0.55, skip_thr=0.001)

# -----------------------------
# Main loop
# -----------------------------
for img_path in image_paths:
    name = os.path.splitext(os.path.basename(img_path))[0]
    outdir = ensure_dir(os.path.join(OUT_BASE, name))

    orig_bgr = cv2.imread(img_path)
    if orig_bgr is None:
        print("Skip (cannot read):", img_path)
        continue

    # 1) Hybrid prediction (WBF fused)
    res = model.predict(source=img_path, imgsz=640, conf=0.25, device=0, verbose=False)[0]
    if res.boxes is None or len(res.boxes) == 0:
        pred_drawn = orig_bgr.copy()
    else:
        pred_drawn = draw_yolo_boxes(
            orig_bgr,
            res.boxes.xyxy,
            res.boxes.conf,
            res.boxes.cls,
            names=res.names
        )

    # 2) Load existing Hybrid Grad-CAM++ overlay (if exists)
    cam_overlay_path = os.path.join(CAM_DIR_EXISTING, f"{name}_HYBRID_gradcampp.jpg")
    cam_overlay_bgr = cv2.imread(cam_overlay_path) if os.path.exists(cam_overlay_path) else None

    # Save base
    save_img(os.path.join(outdir, "01_original.jpg"), orig_bgr)
    save_img(os.path.join(outdir, "02_pred_annotated.jpg"), pred_drawn)

    # 3) CAM-based visualizations (if CAM exists)
    if cam_overlay_bgr is not None:
        save_img(os.path.join(outdir, "03_hybrid_gradcampp_overlay.jpg"), cam_overlay_bgr)

        o = cv2.resize(orig_bgr, (cam_overlay_bgr.shape[1], cam_overlay_bgr.shape[0]))
        diff = cv2.absdiff(cam_overlay_bgr, o)
        diff_gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY).astype(np.float32)
        cam_approx = normalize_0_1(diff_gray)

        hm_bgr = heatmap_bgr_from_cam(cam_approx)
        save_img(os.path.join(outdir, "04_cam_heatmap_only.jpg"), hm_bgr)

        plt.figure(figsize=(6,5))
        plt.imshow(cam_approx, cmap="jet")
        plt.colorbar()
        plt.title("Hybrid CAM intensity (approx)")
        plt.axis("off")
        save_matplotlib_figure(os.path.join(outdir, "05_cam_with_colorbar.png"))

        mask = (threshold_mask(cam_approx, thr=0.6) * 255).astype(np.uint8)
        save_img(os.path.join(outdir, "06_cam_threshold_mask.jpg"), mask)

        cont = contour_overlay(o, cam_approx, levels=(0.4, 0.6, 0.8))
        save_img(os.path.join(outdir, "07_cam_contours.jpg"), cont)

        # draw boxes on CAM overlay
        if res.boxes is not None and len(res.boxes) > 0:
            cam_bbox = draw_yolo_boxes(
                cam_overlay_bgr,
                res.boxes.xyxy,
                res.boxes.conf,
                res.boxes.cls,
                names=res.names,
                color=(0,255,255)
            )
        else:
            cam_bbox = cam_overlay_bgr.copy()
        save_img(os.path.join(outdir, "08_cam_overlay_with_bboxes.jpg"), cam_bbox)

        rgb_orig = cv2.cvtColor(orig_bgr, cv2.COLOR_BGR2RGB)
        rgb_pred = cv2.cvtColor(pred_drawn, cv2.COLOR_BGR2RGB)
        rgb_cam  = cv2.cvtColor(cam_overlay_bgr, cv2.COLOR_BGR2RGB)
        montage3([rgb_orig, rgb_pred, rgb_cam],
                 ["Original", "Hybrid Prediction", "Hybrid Grad-CAM++"],
                 os.path.join(outdir, "09_montage.png"))

        cy, cx = np.unravel_index(np.argmax(cam_approx), cam_approx.shape)
        H, W = o.shape[:2]
        crop_sz = min(H, W) // 2
        y1, y2 = max(0, cy - crop_sz//2), min(H, cy + crop_sz//2)
        x1, x2 = max(0, cx - crop_sz//2), min(W, cx + crop_sz//2)
        crop = o[y1:y2, x1:x2]
        save_img(os.path.join(outdir, "10_top_activation_crop.jpg"), crop)

    # 4) Occlusion sensitivity (faithfulness) using HYBRID score
    occ = occlusion_sensitivity_map(model, orig_bgr, patch=64, stride=64, imgsz=640, conf=0.25, device=0)
    occ_hm = heatmap_bgr_from_cam(occ)
    occ_overlay = cv2.addWeighted(orig_bgr, 0.55, occ_hm, 0.45, 0)
    save_img(os.path.join(outdir, "11_occlusion_heatmap.jpg"), occ_hm)
    save_img(os.path.join(outdir, "12_occlusion_overlay.jpg"), occ_overlay)

    # 5) Deletion & insertion curves using occlusion as importance
    fracs, del_s, ins_s = deletion_insertion_curves(model, orig_bgr, occ, steps=20, imgsz=640, conf=0.25, device=0)
    plt.figure(figsize=(6,4))
    plt.plot(fracs, del_s, label="Deletion (remove important)")
    plt.plot(fracs, ins_s, label="Insertion (add important)")
    plt.xlabel("Fraction of pixels")
    plt.ylabel("Top detection score")
    plt.title("Faithfulness Curves (Hybrid YOLO)")
    plt.legend()
    save_matplotlib_figure(os.path.join(outdir, "13_deletion_insertion_curves.png"))

    print("✅ Saved all explainability outputs to:", outdir)

print("DONE. Master folder:", OUT_BASE)

✅ Saved all explainability outputs to: /content/hybrid_out/explainability_plots/emotion_female
✅ Saved all explainability outputs to: /content/hybrid_out/explainability_plots/emotion_male
DONE. Master folder: /content/hybrid_out/explainability_plots


In [ ]:
from google.colab import files
import shutil

# Replace 'your_folder' with the folder you want to download
folder_name = "/content/hybrid_out"
zip_name = f"{folder_name}.zip"

# Zip the folder
shutil.make_archive(folder_name, 'zip', folder_name)

# Download the zipped folder
files.download(zip_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
import shutil

# Replace 'your_folder' with the folder you want to download
folder_name = "/content/outputs"
zip_name = f"{folder_name}.zip"

# Zip the folder
shutil.make_archive(folder_name, 'zip', folder_name)

# Download the zipped folder
files.download(zip_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>